In [12]:
import sys
from pathlib import Path
notebook_dir = Path().resolve()
src_path = notebook_dir.parent / 'src'
sys.path.insert(0, str(src_path))
import pandas as pd
from utils.timesplit import RollingSplit

crypto = pd.read_csv("../data/processed_data/BTCUSDT_1h_processed.csv")

# Variables
input_features = ['open',
                  'high', 
                  'low', 
                  'close']
train_size = 0.7
val_size = 0.15

# Features and target
X = crypto[input_features]
y = crypto['log_return']

# Initialize Rolling Splitter
sample = len(crypto)
train_count = int(train_size * sample)          # number of samples in the training set
val_count = int(val_size * sample)              # number of samples in the validation set
test_count = sample - train_count - val_count   # number of samples in the test set
step_size = test_count                          # step size as test arbitrarily chosen as test size for non-overlapping sets

splitter = RollingSplit(train_size=train_count, val_size=val_count, test_size=test_count, step_size=step_size)